##### Installation

In [ ]:
!pip install torch
#!pip install tensorflow
#!pip install tensorflow-datasets
!pip install transformers

# for tracking and visualising training in web browser
!pip install wandb
!wandb login 20983d37439737ea3e750cd826d32f9ed826c1dc

  Using cached https://files.pythonhosted.org/packages/14/e5/a26eb4716523808bb0a799fcfdceb6ebf77a18169d9591b2f46a9adb87d9/tokenizers-0.7.0-cp36-cp36m-manylinux1_x86_64.whl
  Found existing installation: tokenizers 0.8.0rc3
    Uninstalling tokenizers-0.8.0rc3:
      Successfully uninstalled tokenizers-0.8.0rc3
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


In [ ]:
%tensorflow_version 2.x
from datetime import datetime
from google.colab import files

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
data_file_name = "data_split_original.txt"

In [ ]:
%mkdir /content/data
%cp -av /gdrive/My\ Drive/data/$data_file_name /content/data/

mkdir: cannot create directory ‘/content/data’: File exists
'/gdrive/My Drive/data/data_split_original.txt' -> '/content/data/data_split_original.txt'


In [ ]:
!ls

data  runs  sample_data  train	wandb


In [ ]:
!nvidia-smi

Sat Jun 27 11:07:24 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

###### Tokenizer

In [ ]:
# Install `transformers` from master
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.11.0
# tokenizers version at notebook update --- 0.8.0rc1

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-shh116a8
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-shh116a8
  Created wheel for transformers: filename=transformers-2.11.0-cp36-none-any.whl size=743047 sha256=b7b020c2a1f0c07b0ae0d059e4e197dbf58e65ade815afa017ec9bc90d45708c
  Stored in directory: /tmp/pip-ephem-wheel-cache-0ea3s3im/wheels/70/d3/52/b3fa4f8b8ef04167ac62e5bb2accb62ae764db2a378247490e
Successfully built transformers
tokenizers               0.8.0rc3       
transformers             2.11.0         


In [ ]:
from pathlib import Path

from tokenizers import BertWordPieceTokenizer

paths = [str(x) for x in Path(".").glob("**/data/data_condense_original.txt")]
print(paths)

# Initialize a tokenizer
tokenizer = BertWordPieceTokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=10000, min_frequency=2, special_tokens=[
    "[CLS]",
    "[SEP]",
    "[UNK]",
    "[PAD]"
])

[]


In [ ]:
!mkdir train
tokenizer.save_model("train")

mkdir: cannot create directory ‘train’: File exists


['train/vocab.txt']

In [ ]:
from google.colab import files
files.download('./train/vocab.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

###### Encode

In [ ]:
from transformers import TransfoXLTokenizerFast

tokenizer = TransfoXLTokenizerFast.from_pretrained('transfo-xl-wt103')
special_tokens_dict = {'cls_token':'[CLS]', 'sep_token':'[SEP]', 'unk_token':'[UNK]', 'pad_token':'[PAD]'}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)


print(tokenizer.encode("start tempo120 melody v80 A4 wait:40"))
print(tokenizer.tokenize("start tempo120 melody v80 A4 wait:40"))

[675, 24, 4934, 24, 28937, 24]
['start', 'tempo120', 'melody', 'v80', 'A4', 'wait:40']


In [ ]:
%mkdir train
%cp -av /gdrive/My\ Drive/data/vocab.json /content/train/

mkdir: cannot create directory ‘train’: File exists
'/gdrive/My Drive/data/vocab.json' -> '/content/train/vocab.json'


In [ ]:
from tokenizers.implementations import BertWordPieceTokenizer
from tokenizers.processors import BertProcessing
from transformers import TransfoXLTokenizerFast

tokenizer = TransfoXLTokenizerFast.from_pretrained(
    "./train",
    # "./train/merges.txt",
    unk_token="[UNK]", sep_token="[SEP]", cls_token="[CLS]", pad_token="[PAD]",
    additional_special_tokens=["[SEP]","[CLS]","[PAD]"]
)

In [ ]:
tokenizer.tokenize("start tempo120 melody v80 A4 wait:40")

['start', 'tempo120', 'melody', 'v80', 'A4', 'wait:40']

In [ ]:
tokenizer.encode("start tempo120 melody v80 A4 wait:40")

[4, 137, 6, 298, 427, 558]

###### Model

In [ ]:
from transformers import TransfoXLConfig
# TransfoXLConfig
config = TransfoXLConfig(
    vocab_size=200000,
    #max_position_embeddings=514, # maximum generation ever
    n_head=12,
    n_layer=6,
    type_vocab_size=1,
    unk_token='[UNK]',
    cls_token='[CLS]',
    sep_token='[SEP]',
    pad_token='[PAD]',
    #vocab_size=50_000,
    #layer_norm_epsilon=int(1e-12),
)

In [ ]:
from transformers import TransfoXLTokenizerFast

tokenizer = TransfoXLTokenizerFast.from_pretrained(
    "./train",
    unk_token="[UNK]", sep_token="[SEP]", cls_token="[CLS]", pad_token="[PAD]",
    additional_special_tokens=["[SEP]","[CLS]","[PAD]"]
)

In [ ]:
# # using Roberta (BERT) for Byte-level BPE since TransfoXLTokenizerFast is word-level

# from transformers import RobertaTokenizerFast, BertTokenizerFast
# # TransfoXLTokenizerFast, RobertaTokenizerFast
# tokenizer = BertTokenizerFast.from_pretrained("./train") #max_len=512

In [ ]:
from transformers import TransfoXLLMHeadModel

# initialise from config, not pretrained model
model = TransfoXLLMHeadModel(config=config)

In [ ]:
from transformers import TransfoXLModel

# initialise from config, not pretrained model
model = TransfoXLModel(config=config)

In [ ]:
model.config.vocab_size

200000

In [ ]:
model.num_parameters()
# => 113.6 # 84 million parameters

112473411

In [ ]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./data/data_split_original.txt",
    block_size=128,
)

# block_size
# 64: 19min 10s
# 32: 19min 27s
# 16: 19min 3s

CPU times: user 36min 25s, sys: 6.07 s, total: 36min 31s
Wall time: 19min 3s


In [ ]:
dataset[0]

tensor([5628,  145, 5646,    6,  309,  415,    6,  309,  408,    6,  309,  421,
           7,  310,  379,  614,    6,  218,  415,    6,  218,  408,    6,  218,
         421,    7,  218,  379,    7,  310,  361,  614,    6,  309,  421,    6,
         309,  415,    6,  309,  408,    7,  218,  361,  566,    6,  218,  408,
           6,  218,  421,    6,  218,  415,    6,  309,  415,    6,  309,  408,
           6,  309,  424,  566])

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False, mlm_probability=0
)

In [ ]:
print(model.config)

TransfoXLConfig {
  "adaptive": true,
  "attn_type": 0,
  "clamp_len": 1000,
  "cls_token": "[CLS]",
  "cutoffs": [
    20000,
    40000,
    200000
  ],
  "d_embed": 1024,
  "d_head": 64,
  "d_inner": 4096,
  "d_model": 1024,
  "div_val": 4,
  "dropatt": 0.0,
  "dropout": 0.1,
  "eos_token_id": 0,
  "ext_len": 0,
  "init": "normal",
  "init_range": 0.01,
  "init_std": 0.02,
  "layer_norm_epsilon": 1e-05,
  "mem_len": 1600,
  "model_type": "transfo-xl",
  "n_head": 12,
  "n_layer": 6,
  "pad_token": "[PAD]",
  "pre_lnorm": false,
  "proj_init_std": 0.01,
  "same_length": true,
  "sample_softmax": -1,
  "sep_token": "[SEP]",
  "tgt_len": 128,
  "tie_projs": [
    false,
    true,
    true,
    true
  ],
  "tie_weight": true,
  "type_vocab_size": 1,
  "unk_token": "[UNK]",
  "untie_r": true,
  "vocab_size": 267735
}



In [ ]:
%%time
from transformers import Trainer, TrainingArguments
import torch

with torch.no_grad():
  training_args = TrainingArguments(
      output_dir="./train",
      overwrite_output_dir=True,
      num_train_epochs=1,
      per_gpu_train_batch_size=8,
      save_steps=10_000,
      save_total_limit=2,
  )

  trainer = Trainer(
      model=model,
      args=training_args,
      data_collator=data_collator,
      train_dataset=dataset,
      prediction_loss_only=True,
  )

RuntimeError: ignored

In [ ]:
import torch
torch.cuda.empty_cache()


In [ ]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


RuntimeError: ignored

In [ ]:
trainer.save_model('./train')

###### TF Train


In [ ]:
# Prepare training: Compile tf.keras model with optimizer, loss and learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [ ]:
# Fix to TypeError: 
# You are attempting to use Python control flow in a layer that was not declared to be dynamic. Pass `dynamic=True` to the class constructor
model.run_eagerly = True

In [ ]:
history = model.run(train_data, epoch=2, )

###### Generation (Pipeline)

In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./train",
    tokenizer="./train"
)

fill_mask("start tempo120 <mask>")

/usr/local/lib/python3.6/dist-packages/transformers/modeling_auto.py:791: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


[{'score': 0.10992492735385895,
  'sequence': '<s>start tempo120:</s>',
  'token': 30,
  'token_str': ':'},
 {'score': 0.00857906136661768,
  'sequence': '<s>start tempo120v</s>',
  'token': 90,
  'token_str': 'v'},
 {'score': 0.0022426971700042486,
  'sequence': '<s>start tempo1200</s>',
  'token': 20,
  'token_str': '0'},
 {'score': 0.0019350973889231682,
  'sequence': '<s>start tempo120 melody</s>',
  'token': 266,
  'token_str': 'Ġmelody'},
 {'score': 0.0008917800732888281,
  'sequence': '<s>start tempo120100</s>',
  'token': 274,
  'token_str': '100'}]

In [ ]:
from transformers import pipeline

text_generator = pipeline(
    "text-generation",
    model="./train",
    tokenizer="./train"
)

text_generator("start tempo120", max_length=1000)

/usr/local/lib/python3.6/dist-packages/transformers/modeling_auto.py:791: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


NotImplementedError: ignored

##### Training


In [ ]:
import tensorflow as tf
import tensorflow_datasets
from transformers import *

# Load dataset, tokenizer, model from pretrained model/vocabulary
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-cased')
data = tensorflow_datasets.load('glue/mrpc')

# Prepare dataset for GLUE as a tf.data.Dataset instance
train_dataset = glue_convert_examples_to_features(data['train'], tokenizer, max_length=128, task='mrpc')
valid_dataset = glue_convert_examples_to_features(data['validation'], tokenizer, max_length=128, task='mrpc')
train_dataset = train_dataset.shuffle(100).batch(32).repeat(2)
valid_dataset = valid_dataset.batch(64)

# Prepare training: Compile tf.keras model with optimizer, loss and learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# Train and evaluate using tf.keras.Model.fit()
history = model.fit(train_dataset, epochs=2, steps_per_epoch=115,
                    validation_data=valid_dataset, validation_steps=7)

# Load the TensorFlow model in PyTorch for inspection
model.save_pretrained('./data/')
pytorch_model = BertForSequenceClassification.from_pretrained('./data/', from_tf=True)

# Quickly test a few predictions - MRPC is a paraphrasing task, let's see if our model learned the task
sentence_0 = "This research was consistent with his findings."
sentence_1 = "His findings were compatible with this research."
sentence_2 = "His findings were not compatible with this research."
inputs_1 = tokenizer.encode_plus(sentence_0, sentence_1, add_special_tokens=True, return_tensors='pt')
inputs_2 = tokenizer.encode_plus(sentence_0, sentence_2, add_special_tokens=True, return_tensors='pt')

pred_1 = pytorch_model(inputs_1['input_ids'], token_type_ids=inputs_1['token_type_ids'])[0].argmax().item()
pred_2 = pytorch_model(inputs_2['input_ids'], token_type_ids=inputs_2['token_type_ids'])[0].argmax().item()

print("sentence_1 is", "a paraphrase" if pred_1 else "not a paraphrase", "of sentence_0")
print("sentence_2 is", "a paraphrase" if pred_2 else "not a paraphrase", "of sentence_0")

INFO:absl:Load pre-computed datasetinfo (eg: splits) from bucket.
INFO:absl:Loading info from GCS for glue/mrpc/1.0.0
INFO:absl:Field info.description from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.location from disk and from code do not match. Keeping the one from code.
INFO:absl:Generating dataset glue (/root/tensorflow_datasets/glue/mrpc/1.0.0)


INFO:absl:Downloading https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2Fmrpc_dev_ids.tsv?alt=media&token=ec5c0836-31d5-48f4-b431-7480817f1adc into /root/tensorflow_datasets/downloads/fire.goog.com_v0_b_mtl-sent-repr.apps.com_o_2FjSIMlCiqs1QSmIykr4IRPnEHjPuGwAz5i40v8K9U0Z8.tsvalt=media&token=ec5c0836-31d5-48f4-b431-7480817f1adc.tmp.1e955c75eca444c7801f9ffbb7d945c8...
INFO:absl:Downloading https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_train.txt into /root/tensorflow_datasets/downloads/dl.fbaip.com_sente_sente_msr_parap_trainfGxPZuQWGBti4Tbd1YNOwQr-OqxPejJ7gcp0Al6mlSk.txt.tmp.4b3a795fb50d40c0a2149a450a36301d...
INFO:absl:Downloading https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_test.txt into /root/tensorflow_datasets/downloads/dl.fbaip.com_sente_sente_msr_parap_test0PdekMcyqYR-w4Rx_d7OTryq0J3RlYRn4rAMajy9Mak.txt.tmp.5608fc42a1e542cdbc4b51715f845c3f...
/usr/local/lib/python3.6/dist-packages/urllib3/co

Shuffling and writing examples to /root/tensorflow_datasets/glue/mrpc/1.0.0.incompleteJTKUVU/glue-train.tfrecord


INFO:absl:Done writing /root/tensorflow_datasets/glue/mrpc/1.0.0.incompleteJTKUVU/glue-train.tfrecord. Shard lengths: [3668]
INFO:absl:Generating split validation


Shuffling and writing examples to /root/tensorflow_datasets/glue/mrpc/1.0.0.incompleteJTKUVU/glue-validation.tfrecord


INFO:absl:Done writing /root/tensorflow_datasets/glue/mrpc/1.0.0.incompleteJTKUVU/glue-validation.tfrecord. Shard lengths: [408]
INFO:absl:Generating split test


Shuffling and writing examples to /root/tensorflow_datasets/glue/mrpc/1.0.0.incompleteJTKUVU/glue-test.tfrecord


INFO:absl:Done writing /root/tensorflow_datasets/glue/mrpc/1.0.0.incompleteJTKUVU/glue-test.tfrecord. Shard lengths: [1725]
INFO:absl:Skipping computing stats for mode ComputeStatsMode.AUTO.
INFO:absl:Constructing tf.data.Dataset for split None, from /root/tensorflow_datasets/glue/mrpc/1.0.0


Dataset glue downloaded and prepared to /root/tensorflow_datasets/glue/mrpc/1.0.0. Subsequent calls will reuse this data.
Epoch 1/2
115/115 [==============================] - 61s 534ms/step - loss: 0.5476 - accuracy: 0.7331 - val_loss: 0.4119 - val_accuracy: 0.8162
Epoch 2/2
115/115 [==============================] - 59s 509ms/step - loss: 0.2754 - accuracy: 0.8918 - val_loss: 0.4856 - val_accuracy: 0.8235
sentence_1 is a paraphrase of sentence_0
sentence_2 is a paraphrase of sentence_0


In [ ]:
!mkdir train

In [ ]:
from transformers import TransfoXLTokenizer, TFTransfoXLModel
import torch
import tensorflow as tf
import tensorflow_datasets as tfds

import os

In [ ]:
from transformers import TransfoXLTokenizer, TFTransfoXLModel
import torch
import tensorflow as tf
import tensorflow_datasets as tfds

import os

###### Convert .txt file to tf.data.Dataset

In [ ]:
file_name = 'data/data_express_original.txt'

In [ ]:
pretrained_weight = 'transfo-xl-wt103'
tokenizer = TransfoXLTokenizer.from_pretrained(pretrained_weight)
model = TFTransfoXLModel.from_pretrained(pretrained_weight)

In [ ]:
from pathlib import Path

paths = [str(x) for x in Path(".").glob("data/data_express_transpose_original.txt")] #glob("data/*.txt")
print(paths)

tokenizer.train(files=paths)

[]


TypeError: ignored

In [ ]:
from tokenizers import Tokenizer
from tokenizers.decoders import ByteLevel as ByteLevelDecoder
from tokenizers.models import BPE
from tokenizers.normalizers import Lowercase, NFKC, Sequence
from tokenizers.pre_tokenizers import ByteLevel

# First we create an empty Byte-Pair Encoding model (i.e. not trained model)
tokenizer = Tokenizer(BPE())

# Then we enable lower-casing and unicode-normalization
# The Sequence normalizer allows us to combine multiple Normalizer that will be
# executed in order.
tokenizer.normalizer = Sequence([
    NFKC(),
    Lowercase()
])

# Our tokenizer also needs a pre-tokenizer responsible for converting the input to a ByteLevel representation.
tokenizer.pre_tokenizer = ByteLevel()

# And finally, let's plug a decoder so we can recover from a tokenized input to the original one
tokenizer.decoder = ByteLevelDecoder()


from tokenizers.trainers import BpeTrainer

# We initialize our trainer, giving him the details about the vocabulary we want to generate
trainer = BpeTrainer(vocab_size=25000, show_progress=True, initial_alphabet=ByteLevel.alphabet())
tokenizer.train(trainer, ["./data/data_express_original.txt"])

print("Trained vocab size: {}".format(tokenizer.get_vocab_size()))

Trained vocab size: 521


In [ ]:
# You will see the generated files in the output.
tokenizer.model.save('./train/')

['./train/vocab.json', './train/merges.txt']

In [ ]:
# Let's tokenizer a simple input
tokenizer.model = BPE('./train/vocab.json', './train/merges.txt')
encoding = tokenizer.encode("[CLS] start tempo120 melody:v100:A4 wait:12")

print("Encoded string: {}".format(encoding.tokens))

decoded = tokenizer.decode(encoding.ids)
print("Decoded string: {}".format(decoded))

Encoded string: ['Ġ[', 'cls', ']', 'Ġ', 's', 't', 'a', 'r', 't', 'Ġtempo', '120', 'Ġmelody', ':', 'v', '100', ':', 'a', '4', 'Ġwait', ':', '12']
Decoded string:  [cls] start tempo120 melody:v100:a4 wait:12


In [ ]:
def labeler(example, index):
  return example, tf.cast(index, tf.int64)  

labeled_data_sets = []

lines_dataset = tf.data.TextLineDataset(file_name)
labeled_dataset = lines_dataset.map(lambda ex: labeler(ex, 0))
labeled_data_sets.append(labeled_dataset)

# for i, file_name in enumerate(FILE_NAMES):
#   lines_dataset = tf.data.TextLineDataset(os.path.join(parent_dir, file_name))
#   labeled_dataset = lines_dataset.map(lambda ex: labeler(ex, i))
#   labeled_data_sets.append(labeled_dataset)

In [ ]:
BUFFER_SIZE = 50000
BATCH_SIZE = 64
TAKE_SIZE = 5000

all_labeled_data = labeled_data_sets[0]

# all_labeled_data = labeled_data_sets[0]
# for labeled_dataset in labeled_data_sets[1:]:
#   all_labeled_data = all_labeled_data.concatenate(labeled_dataset)
  
shuffled_labeled_data = all_labeled_data.shuffle(BUFFER_SIZE, reshuffle_each_iteration=False)

In [ ]:
# example
for ex in all_labeled_data.take(5):
  print(ex)

In [ ]:
tokenizer = tfds.features.text.Tokenizer()

vocabulary_set = set()
for text_tensor, _ in all_labeled_data:
  some_tokens = tokenizer.tokenize(text_tensor.numpy())
  vocabulary_set.update(some_tokens)

vocab_size = len(vocabulary_set)

vocab_size

print(vocabulary_set)

{'55', '135', '12', 'v78', 'accomp10', '111', '139', '38', 'F2', '1504', 'v48', '120', '21', 'v65', '79', 'v83', '110', 'v99', '96', '209', '125', 'v59', 'tempo91', '87', 'G4', '39', '767', 'v75', 'v89', '948', '411', '26', 'tempo126', '76', 'v63', 'v91', '11', 'v66', '100', '46', 'tempo86', '3875', '16', '205', '105', '13', 'tempo170', '455', 'v77', 'v7', 'tempo60', '41', 'v25', 'v71', '20', '298', 'melody', 'v97', '440', 'C3', 'tempo147', 'F4', '113', 'C5', '314', '102', '186', 'G3', 'v8', '45', '70', '960', '121', 'B4', 'D7', 'v53', '3632', '122', 'v32', 'v30', '104', '37', '71', 'E6', '1', 'tempo115', 'tempo175', 'C4', 'accomp6', '89', 'B', '815', '40', '95', '50', '577', '67', 'v88', 'v54', '181', '42', '151', 'v33', '74', '3647', 'v100', '996', 'v47', 'tempo87', 'tempo148', '303', 'v69', '866', '86', '191', '144', '22', '19', 'v18', '24', '383', 'tempo150', 'v60', 'tempo107', '69', 'v74', 'tempo104', 'tempo109', '10898', 'accomp1', 'C7', 'v119', '896', '345', '118', '864', '30', 

In [ ]:
encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)

In [ ]:
# example
example_text = next(iter(all_labeled_data))[0].numpy()
print(example_text)

b'[CLS] tempo128 [127] melody:v91:D4 melody:v91:B-3 melody:v91:F4 accomp0:v92:F2 wait:96 melody:v0:D4 melody:v0:B-3 melody:v0:F4 accomp0:v0:F2 accomp0:v92:G1 wait:96 melody:v91:F4 melody:v91:D4 melody:v91:B-3 accomp0:v0:G1 wait:48 melody:v0:B-3 melody:v0:F4 melody:v0:D4 melody:v91:D4 melody:v91:B-3 melody:v91:G4 wait:48 melody:v0:G4 melody:v0:B-3 melody:v0:D4 wait:24 melody:v91:D4 melody:v91:B-3 melody:v91:G4 wait:48 melody:v0:D4 melody:v0:B-3 melody:v0:G4 wait:24 melody:v91:B-3 melody:v91:D4 melody:v91:F4 accomp0:v92:F2 wait:96 [126] melody:v0:B-3 melody:v0:D4 melody:v0:F4 accomp0:v0:F2 accomp0:v92:G1 wait:96 melody:v91:F4 melody:v91:B-3 melody:v91:D4 accomp0:v0:G1 wait:48 melody:v0:D4 melody:v0:F4 melody:v0:B-3 melody:v91:D4 melody:v91:B-3 melody:v91:G4 wait:48 melody:v0:G4 melody:v0:B-3 melody:v0:D4 wait:24 melody:v91:D4 melody:v91:B-3 melody:v91:G4 wait:48 melody:v0:D4 melody:v0:B-3 melody:v0:G4 wait:24 melody:v91:B-3 melody:v91:D4 melody:v91:F4 accomp0:v92:F2 wait:96 melody:v0:B-3

In [ ]:
def encode(text_tensor, label):
  encoded_text = encoder.encode(text_tensor.numpy())
  return encoded_text, label

In [ ]:
def encode_map_fn(text, label):
  # py_func doesn't set the shape of the returned tensors.
  encoded_text, label = tf.py_function(encode, 
                                       inp=[text, label], 
                                       Tout=(tf.int64, tf.int64))

  # `tf.data.Datasets` work best if all components have a shape set
  #  so set the shapes manually: 
  encoded_text.set_shape([None])
  label.set_shape([])

  return encoded_text, label


all_encoded_data = all_labeled_data.map(encode_map_fn)

In [ ]:
# example
example_text = next(iter(all_encoded_data))[0].numpy()
print(example_text)

[192 236 288 ... 273 188 143]


In [ ]:
train_data = all_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
train_data = train_data.padded_batch(BATCH_SIZE).repeat(30)

test_data = all_encoded_data.take(TAKE_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE)

In [ ]:
test_data

<PaddedBatchDataset shapes: ((None, None), (None,)), types: (tf.int64, tf.int64)>

In [ ]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./data/data_express_original.txt",
    block_size=128,
)

AttributeError: ignored

###### Training

In [ ]:
# Prepare training: Compile tf.keras model with optimizer, loss and learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# Fix to TypeError: 
# You are attempting to use Python control flow in a layer that was not declared to be dynamic. Pass `dynamic=True` to the class constructor
model.run_eagerly = True

In [ ]:
# Train and evaluate using tf.keras.Model.fit()
history = model.fit(train_data, epochs=2) #steps_per_epoch=115,)
                    #validation_data=test_data, validation_steps=7)

Epoch 1/2


TypeError: ignored

In [ ]:
eval_loss, eval_acc = model.evaluate(test_data)
print('\nEval loss: {:.3f}, Eval accuracy: {:.3f}'.format(eval_loss, eval_acc))

In [ ]:
model.save_pretrained('./save/')

In [ ]:
model = tf.keras.Sequential()

# The first layer converts integer representations to dense vector embeddings
model.add(tf.keras.layers.Embedding(vocab_size, 64))

# The next layer is a Long Short-Term Memory layer, 
# which lets the model understand words in their context with other words. 
# A bidirectional wrapper on the LSTM helps it to learn about the datapoints 
# in relationship to the datapoints that came before it and after it.
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))

# a series of one or more densely connected layers, with the last one being the output layer. 
# The output layer produces a probability for all the labels. 
# The one with the highest probability is the models prediction of an example's label.
for units in [64, 64]:
  model.add(tf.keras.layers.Dense(units, activation='relu'))

# Output layer. The first argument is the number of labels.
model.add(tf.keras.layers.Dense(3))


model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(train_data, epochs=3, validation_data=test_data)

Epoch 1/3


TypeError: ignored

In [ ]:


special_tokens_dict = {'cls_token': '[CLS]',
                      'sep_token': '[SEP]',
                      }


num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
print('Added new tokens: {}'.format(num_added_toks))
model.resize_token_embeddings(len(tokenizer))

train

In [ ]:
input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True)).unsqueeze(0)  # Batch size 1
outputs = model(input_ids)
last_hidden_states, mems = outputs[:2]